In [ ]:
import tensorflow as tf
import os
import json

In [ ]:
os.environ['TF_CONFIG'] = json.dumps(
{
  "cluster": {
    "coordinator": ["192.168.2.1:2223"],
    "ps": ["192.168.2.2:2223"],
    "worker": ["192.168.2.3:2223", "192.168.2.4:2223"]
  },
  "task": {
    "type": "coordinator",
    "index": 0
  }
})
os.environ["grpc_fail_fast"]="use_caller"

# Create a TensorFlow ClusterSpec
cluster_spec = tf.train.ClusterSpec({
    "coordinator": ["192.168.2.1:2223"],
    "ps": ["192.168.2.2:2223"],
    "worker": ["192.168.2.3:2223", "192.168.2.4:2223"]
})

# Start a TensorFlow server
resolver = tf.distribute.cluster_resolver.SimpleClusterResolver(cluster_spec=cluster_spec)
server = tf.distribute.Server(cluster_spec, job_name="coordinator", task_index=0)

In [ ]:
strategy = tf.distribute.ParameterServerStrategy(resolver)

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=10000).batch(64)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(64)

In [ ]:
with strategy.scope():
    asynchronous_model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
        ])  # Make sure the `Model` is created within scope.
    
asynchronous_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", steps_per_execution=10)

In [ ]:
asynchronous_model.fit(train_dataset, epochs=10, steps_per_epoch=20, validation_data=test_dataset)